方法1：余弦相似度直接计算

In [40]:
from text2vec import SentenceModel, semantic_search
import sentence_transformers
import torch
import numpy as np
import os
#文件读取
document_name = 'somuut'
document_path='./data/'+document_name+'.txt'
vector_path = './data/'+document_name+'.pt'
with open(document_path, 'r', encoding='utf-8') as file:
    document = file.readlines()
document = [line.strip() for line in document]
sequence_id = np.arange(0, len(document)) 
embedder  = SentenceModel('shibing624/text2vec-base-chinese')
if os.path.exists(vector_path):
    embeddings = torch.load(vector_path)
    print("already load vector")
else:
    embeddings = embedder.encode(document, convert_to_tensor=True)
    torch.save(embeddings, vector_path)
    
def ranking_api(query,document,embedder,embeddings,top_k):
    if not query: 
        zero_score = [0.0 for i in range(0,len(document))]
        return document,zero_score
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    sim_scores = sentence_transformers.util.cos_sim(query_embedding, embeddings)[0]
    #对得分进行排序
    # top_k = len(document)
    top_k_cat = torch.topk(sim_scores, k=top_k)
    top_k_score,top_k_idx = top_k_cat[0],top_k_cat[1]
    top_k_name = [document[top_k_idx[i]] for i in range(0,top_k)]
    return top_k_name,top_k_score,top_k_idx

2024-05-30 21:21:08.115 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda


In [41]:
query= "天然气管道巡检有哪些标准"
ranking_api(query,document,embedder,embeddings,top_k=10)

(['天然气管道应按巡检计划定期巡检，运行状况应符合下列规定：',
  '天然气管道应按巡检计划定期巡检',
  '管道巡检应包括下列内容：',
  '天然气管道巡检计划应根据管线的压力等级、供应用户情况',
  '天然气管道应定期检查放散系统，',
  '天然气管道的运行维护及安全管理尚应符合国家现行标准《城镇燃气设施运行、维护和检修安全技术规程》CJJ 51、《城镇燃气管网泄漏检测技术规程》CJJ/T 215和《燃气系统运行安全评价标准》GB/T 50811的有关规定。',
  '综合管廊内天然气管线的运行维护及安全管理还应符合国家现行标准《城镇燃气设施运行、维护和检修安全技术规程》CJJ 51、《城镇燃气管网泄漏检测技术规程》CJJ 215和《燃气系统运行安全评价标准》GB/T 50811的有关规定。',
  '排水管道的巡检应包括下列内容：',
  '给水排水系统巡检内容及要求应符合表5．7．4的规定。',
  '应定期检查天然气管线放散管的接地可靠性、牢固程度和管道通畅性。'],
 tensor([0.7777, 0.7516, 0.7287, 0.7189, 0.7080, 0.7064, 0.6882, 0.6797, 0.6785,
         0.6779], device='cuda:0'),
 tensor([207,   3, 630, 728, 290, 364, 187, 314,  24, 108], device='cuda:0'))

方法2：faiss检索库

In [5]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle
import numpy as np
import os

def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level
    DistilBERT model and finds similar vectors using FAISS.
 
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
 
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
 
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]

#文件读取
document_name = 'somuut'
document_path='./data/'+document_name+'.txt'
vector_path = './data/'+document_name+'.npy'

with open(document_path, 'r', encoding='utf-8') as file:
    document = file.readlines()
document = [line.strip() for line in document]
sequence_id = np.arange(0, len(document)) 

# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
   model = model.to(torch.device("cuda:0"))
print(model.device)

if os.path.exists(vector_path):
    embeddings = np.load(vector_path)
    print("already load vector")
else:
    # Convert abstracts to vectors
    embeddings = model.encode(document, show_progress_bar=True)
    embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
    np.save(vector_path, embeddings)

index = faiss.IndexFlatL2(embeddings.shape[1])# Step 1: Instantiate the index
index = faiss.IndexIDMap(index)# Step 2: Pass the index to IndexIDMap
index.add_with_ids(embeddings, sequence_id)# Step 3: Add vectors and their IDs

d:\anaconda3\envs\faiss\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


cuda:0
already load vector


In [22]:
user_query = "天然气管道巡检有哪些标准"
# Querying the index
D, I = vector_search([user_query], model, index, num_results=10)
print([document[i] for i in I.tolist()[0]])